In [61]:
import numpy as np # array operations
import cv2 #opencv library read img operations
import os # use directory & join path
from pathlib import Path # 去掉副檔名
import pandas as pd # 匯出 csv
import struct
from skimage import io
from skimage import color
from skimage.util import img_as_ubyte

# data visualisation and manipulation
import matplotlib.pyplot as plt #show img and table

In [62]:
getBin = lambda x: x > 0 and str(bin(x))[2:] or "-" + str(bin(x))[3:]
 
def floatToBinary64(value):
    val = struct.unpack('Q', struct.pack('d', value))[0]
    return getBin(val)
 
def binaryToFloat(value):
    hx = hex(int(value, 2))   
    return struct.unpack("d", struct.pack("q", int(hx, 16)))[0]

def dec_to_bin_csv():
    DATASRC = 'feature/dec/'
    # 抓目錄下所有圖檔檔名
    src_name = os.listdir(DATASRC)
    for i in range (len(src_name)):

        # 檔案完整路徑
        src_path = DATASRC + src_name[i]
        data = pd.read_csv(src_path,header=None)
        
        mean = [floatToBinary64(float(data[1][1])),floatToBinary64(float(data[2][1])),floatToBinary64(float(data[3][1]))]
        std = [floatToBinary64(float(data[1][2])),floatToBinary64(float(data[2][2])),floatToBinary64(float(data[3][2]))]
        weight = [floatToBinary64(float(data[1][3])),floatToBinary64(float(data[2][3])),floatToBinary64(float(data[3][3]))]
        
        MeanStd = [mean,std,weight]
        MeanStd_table = pd.DataFrame(MeanStd)
        col_name = ['red','green','blue']
        row_name = ['Mean','Standard Deviation','weight']
        MeanStd_table.columns = col_name
        MeanStd_table.index = row_name
        # print(MeanStd_table)

        # 匯出 answer table 成 csv 檔
        src_name_temp= Path(src_name[i]).stem 
        src_name_tempp = src_name_temp.rstrip('dec')
        MeanStd_table.to_csv('output/bin/'+ src_name_tempp + 'bin.csv')

In [63]:
def get_mean_and_std(typeStr, img_name, x, x_weight):
    x_mean, x_std = cv2.meanStdDev(x)
    x_mean = np.hstack(np.around(x_mean,2))
    x_std = np.hstack(np.around(x_std,2))
    
    # 印出平均與標準差
    if(typeStr == 's'):
        print("\n來源圖檔名: ", img_name)
    elif(typeStr == 't'):
        print("\n目標圖檔名: ", img_name)
    
    MeanStd = [x_mean,x_std,x_weight]
    MeanStd_table = pd.DataFrame(MeanStd)
    col_name = ['red','green','blue']
    row_name = ['Mean','Standard Deviation','weight']
    MeanStd_table.columns = col_name
    MeanStd_table.index = row_name
    print(MeanStd_table)

    # 匯出 answer table 成 csv 檔
    MeanStd_table.to_csv('feature/dec/'+ img_name + '_dec.csv')
    # 轉成 IEEE bin
    dec_to_bin_csv()
    
    return x_mean, x_std

In [64]:
def color_transfer(src_lab, tar_lab, src_name, tar_name, coltra_name, coltra_num, weight_r, weight_g, weight_b):
    
    x_weight=[weight_r, weight_g, weight_b]
    s_mean, s_std = get_mean_and_std('s', src_name, src_lab, x_weight)
    t_mean, t_std = get_mean_and_std('t', tar_name, tar_lab, x_weight)

    # height, width, channel = src_rgb.shape
    
    coltra_lab = src_lab

    for k in range(0,3):
        x = src_lab[:,:,k]
        x = ((x-s_mean[k])*(((x_weight[k]*t_std[k])+(1-x_weight[k])*s_std[k])/s_std[k]))+ x_weight[k]*t_mean[k]+(1-x_weight[k])*s_mean[k]

        # # round or +0.5
        # x = np.round(x)
        x = np.ceil(x)

        # boundary check 超過邊界拉回在邊界上
        x = np.clip(x, 0, 255)
        coltra_lab[:,:,k] = x
    
    # 存顏色轉換結果圖
    str_weight = '_' + str(weight_r) + '_' + str(weight_g) + '_' + str(weight_b)
    # coltra_bgr = cv2.cvtColor(coltra_lab,cv2.COLOR_Lab2BGR)
    coltra_rgb = color.lab2rgb(coltra_lab)
    coltra_rgb = img_as_ubyte(coltra_rgb)
    print(coltra_rgb.dtype)
    io.imsave('result/'+ coltra_name + str_weight + '.png', coltra_rgb)
    

In [65]:
if __name__ == '__main__':

    weight_r, weight_g, weight_b = input("Enter 3 channel weight_red , weight_green, weight_blue(0.0 ≤ w ≤ 1.0):").split()
    
    weight_r = float(weight_r)
    weight_g = float(weight_g)
    weight_b = float(weight_b)
    
    # print("weight is: " + weight_r, weight_g, weight_b)
    # weight_r, weight_g, weight_b = 0.49,0.54,0.62

    # 印出所有圖片
    DATASRC = 'source/'
    DATATRG = 'target/'

    # 抓目錄下所有圖檔檔名
    src_name = os.listdir(DATASRC)
    tar_name = os.listdir(DATATRG)

    for i in range (len(src_name)):
    # for i in range (1):
  
        # 圖片完整路徑
        src_path = DATASRC + src_name[i]
        tar_path = DATATRG + tar_name[i]
        
        # convert img to array 以彩色格式讀取(三維)
        src_rgb = io.imread(src_path)  
        tar_rgb = io.imread(tar_path)
        src_rgb = img_as_ubyte(src_rgb)
        tar_rgb = img_as_ubyte(tar_rgb)

        src_lab = color.rgb2lab(src_rgb)
        tar_lab = color.rgb2lab(tar_rgb)
        
        # 去掉副檔名
        src_name_temp= Path(src_path).stem 
        tar_name_temp= Path(tar_path).stem 
        tar_name_tempp = tar_name_temp.lstrip('0'+str(i+1))
        coltra_name = src_name_temp + tar_name_tempp
        
        # 做色彩轉換
        color_transfer(src_lab, tar_lab, src_name_temp, tar_name_temp, coltra_name, i+1, weight_r, weight_g, weight_b)



來源圖檔名:  01_kodim17
                      red  green  blue
Mean                32.16   1.12  5.49
Standard Deviation  20.81   3.27  6.75
weight               0.49   0.54  0.62

目標圖檔名:  01_kodim23
                      red  green   blue
Mean                47.09   0.56  20.69
Standard Deviation  18.52  21.61  19.68
weight               0.49   0.54   0.62


c:\Users\yam85\anaconda3\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 4 pixels
  return func(*args, **kwargs)


uint8

來源圖檔名:  02_kodim24
                      red  green  blue
Mean                45.42  -0.98  9.18
Standard Deviation  21.03   5.42  7.79
weight               0.49   0.54  0.62

目標圖檔名:  02_kodim05
                      red  green   blue
Mean                34.58   2.59   9.54
Standard Deviation  20.20  10.71  11.68
weight               0.49   0.54   0.62
uint8

來源圖檔名:  03_kodim07
                      red  green   blue
Mean                46.52  -1.37  15.30
Standard Deviation  15.06  10.02   9.95
weight               0.49   0.54   0.62

目標圖檔名:  03_kodim08
                      red  green  blue
Mean                50.29   2.26  3.57
Standard Deviation  25.03   6.60  8.95
weight               0.49   0.54  0.62
uint8

來源圖檔名:  04_miscanthus
                      red  green  blue
Mean                51.72   0.75  7.73
Standard Deviation  31.62   3.11  8.38
weight               0.49   0.54  0.62

目標圖檔名:  04_sunset
                      red  green   blue
Mean                45.03   4.18

c:\Users\yam85\anaconda3\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 7 pixels
  return func(*args, **kwargs)


uint8

來源圖檔名:  06_horse
                      red  green   blue
Mean                67.61  -2.37  15.40
Standard Deviation  23.07   8.66   7.98
weight               0.49   0.54   0.62

目標圖檔名:  06_hat
                      red  green   blue
Mean                46.54  12.03  14.45
Standard Deviation  21.66  20.50  21.88
weight               0.49   0.54   0.62


c:\Users\yam85\anaconda3\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 23 pixels
  return func(*args, **kwargs)


uint8
